# Data Portal adaptation demo

**This demo implements an analytical workflow that integrates...**
* Climate-hazard probability datasets linked from Google Earth Engine
* Interactivity
* Data outputs compatible with adaptation reporting under the CRF

**What to notice**
* Current and future hazard magnitudes and probabilities are displayed in the table, the graph, and the map
* Define location of interest by placing a circle-marker, drawing a polygon, or importing a GeoJSON describing a point or polygon
* The demo has three sections; be sure to scroll all the way to the right


In [1]:
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
# GEE authentication using service account key stored in local directory
import ee
#service_account = 'climate-hazard-demo@data-portal-adaptation.iam.gserviceaccount.com'
#credentials = ee.ServiceAccountCredentials(service_account, 'data-portal-adaptation.json')
ee.Initialize()

In [3]:
import geemap
from ipyleaflet import CircleMarker, Polygon, LayersControl, DrawControl, ZoomControl
from ipywidgets import interact, interactive, interact_manual, Layout
import ipywidgets as widgets
import asyncio, codecs, json
import numpy as np

In [4]:
# Create decorator to prevent events triggers while slider is being moved
# From https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html
class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

In [5]:
# Load data from GEE
MAPprobs = ee.ImageCollection("users/tedwongwri/MAP_probs/ehe_MAPprob_prime01")
NC_MAPprobs = ee.ImageCollection("users/tedwongwri/MAP_probs/noncumul_ehe_MAPprob_prime01")

In [6]:
initial_threshold = 30
min_threshold = 5
max_threshold = 50
threshold_interval = 5
currentYear = 2022
futureYear = 2050
prev_threshold_current = initial_threshold
prev_threshold_future = initial_threshold
prev_futureYear = 2050

persistent_threshold_value = initial_threshold
persistent_ev_current = None
persistent_ev_future = None
persistent_threshold_value = initial_threshold
initial_coords = [50.85045, 4.34878]  # Brussels
current_geom = ee.Geometry.Point(initial_coords)
current_location_marker = None
ev_val = 'expected'
mode_is_ev = False

In [7]:
futureyear_slider = widgets.IntSlider(
    value = futureYear,
    min = currentYear + 1,
    max = 2080,
    step = 1,
    description = 'Future year',
    continuous_update = False
)

threshold_slider = widgets.IntSlider(
    value=initial_threshold,
    min=5,
    max=50,
    step=5,
    description='EHE ≥ ___',
    continuous_update=False
)
threshold_slider.layout.visibility = ['visible', 'hidden'][mode_is_ev * 1]

table_display = widgets.HTML()
debug_display = widgets.HTML()

mode_toggle = widgets.ToggleButtons(
    options=[ev_val, 'set magnitude'],
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['What EHE can you expect?', 'How probable is a specific EHE level?'],
    value='set magnitude'
)

upload_button = widgets.FileUpload(
    accept = '.geojson',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple = False,  # True to accept multiple files upload else False
    description = 'GeoJSON'
)

In [8]:
currentprobs = MAPprobs.filterMetadata('year', 'equals', currentYear).first()
futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()

In [9]:
Map = geemap.Map(center=initial_coords, zoom=3)
Map.add_basemap('HYBRID')

Map.clear_controls()
layer_control = LayersControl(position='topright')
Map.add_control(layer_control)
Map.add_control(ZoomControl(position='bottomright'))
draw_control = DrawControl(circle={}, polyline={}, rectangle={}, position='topleft')
Map.add_control(draw_control)

current_location_marker = CircleMarker(location=initial_coords, name="location of interest")
Map.add_layer(current_location_marker)

vizParamsC = {
  'bands': ['threshold_' + str(initial_threshold)],
  'min': 0,
  'max': 1,
  'palette': ['000000', '909000'],
  'opacity': 0.5
}
vizParamsF = {
  'bands': ['threshold_' + str(initial_threshold)],
  'min': 0,
  'max': 1,
  'palette': ['000000', 'ff0000'],
  'opacity': 0.5
}

Map.addLayer(currentprobs, vizParamsC, str(currentYear) + ' Prob(EHE ≥ ' + str(initial_threshold) + ')')
Map.addLayer(futureprobs, vizParamsF, str(futureYear) + ' Prob(EHE ≥ ' + str(initial_threshold) + ')')

In [10]:
def expected_value(probs, mags):
    np_probs = np.array(probs)
    np_mags = np.array(mags)
    return np.dot(np_probs, np_mags) / np_probs.size

def get_ev_ehe(year):
# Plot expected value of EHE for given year
    current_NC_probs = NC_MAPprobs.filterMetadata('year', 'equals', year).first().rename([str((i+1)*5) for i in range(25)] )
    current_NC_data = current_NC_probs.reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    dataprobs = [float(i) for i in current_NC_data.values()]
    datamags = [float(i) for i in list(current_NC_data.keys())]
    return expected_value(dataprobs, datamags)

In [11]:
def renameBands(img):
# MAPprob files have bandnames like "threshold_5"
    return img.rename(['05', '10', '15', '20', '25', '30', '35', '40', '45', '50'] )

def nearest_threshold(num):
    return min(max(min_threshold, threshold_interval*(round(num/threshold_interval))), max_threshold)

def prob(threshold, year):
# Returns prob of given EHE threshold for given year, as float betw 0 and 1 inclusive
# Threshold: 5-50 by fives; Year: 2022-2080 by ones    
    data = renameBands(MAPprobs.filterMetadata('year', 'equals', year).first()).reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    return data[['0',''][threshold > 5] + str(threshold)]

def prob_interp(magnitude, year):
# Finds prob by finding magnitude values in the data that are below and above given magnitude
# and treating prob as linear between the two.
    data = NC_MAPprobs.filterMetadata('year', 'equals', year).first().rename([str((i+1)*5) for i in range(25)]).reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    probs = np.array([float(i) for i in data.values()])
    mags = np.array([float(i) for i in data.keys()])
    lte = np.nonzero(mags <= magnitude)[0]
    lte_idx = -1
    if lte.size == 0:
        lte = 0
    else:
        lte_idx = lte[-1]
        lte = mags[lte_idx]
    if lte_idx < mags.size - 2:
        slope = (magnitude - lte) / (mags[lte_idx + 1] - lte)
        if lte_idx >=0:
            return probs[lte_idx] + (slope * (probs[lte_idx + 1] - probs[lte_idx]))
        else:
            probs[0]
    else:
        return probs[-1]

def update_map():
# Updates map w new prob layers according to futureYear and whichever thresholds are appropriate given mode_is_ev state
# Updates some globals
    global prev_futureYear
    global prev_threshold_current
    global prev_threshold_future
    
    Map.remove_ee_layer(name=str(currentYear) + ' Prob(EHE ≥ ' + str(prev_threshold_current) + ')')
    Map.remove_ee_layer(name=str(prev_futureYear) + ' Prob(EHE ≥ ' + str(prev_threshold_future) + ')')
    if mode_is_ev:
        c_threshold = nearest_threshold(persistent_ev_current)
        f_threshold = nearest_threshold(persistent_ev_future)
    else:
        c_threshold = persistent_threshold_value
        f_threshold = persistent_threshold_value
    new_visparamsC = {
      'bands': ['threshold_' + str(c_threshold)],
      'min': 0,
      'max': 1,
      'palette': ['000000', '909000'],
      'opacity': 0.5
    }
    new_visparamsF = {
      'bands': ['threshold_' + str(f_threshold)],
      'min': 0,
      'max': 1,
      'palette': ['000000', 'ff0000'],
      'opacity': 0.5
    }
    Map.add_ee_layer(currentprobs, new_visparamsC, str(currentYear) + ' Prob(EHE ≥ ' + str(c_threshold) + ')')
    Map.add_ee_layer(futureprobs, new_visparamsF, str(futureYear) + ' Prob(EHE ≥ ' + str(f_threshold) + ')')
    prev_threshold_current = c_threshold
    prev_threshold_future = f_threshold

def CRFtext(probstring, prob):
# Assigns label to numerical probability according to Box 4 (p. 44) of
# https://www.globalcovenantofmayors.org/wp-content/uploads/2019/08/Data-TWG_Reporting-Framework_GUIDENCE-NOTE_FINAL.pdf
    if prob > 0.05:
        return 'high'
    elif prob > 0.005:
        return 'moderate'
    elif prob > 0.0005:
        return 'low'
    else:
        return 'do not know'
    
def changetext(before, after, inv):
    if before == after:
        return 'no change'
    chg = after > before
    if inv:
        chg = not chg
    return ['decrease', 'increase'][chg * 1]
    
def update_table():
# Inserts output HTML into table_display widget
# Different HTML depending on whether mode_is_ev
    if mode_is_ev:
        current_ev = persistent_ev_current
        future_ev = persistent_ev_future
        current_prob = prob_interp(current_ev, currentYear)
        future_prob = prob_interp(future_ev, futureYear)
        evstring_current = '{:.1f} days'.format(current_ev)
        evstring_future = '{:.1f} days'.format(future_ev)
    else:
        current_prob = prob(persistent_threshold_value, currentYear)
        future_prob = prob(persistent_threshold_value, futureYear)
    probstring_current = '{:.1f}%'.format(current_prob * 100)
    probstring_current += ' (' + CRFtext(probstring_current, current_prob) + ')'
    probstring_future = '{:.1f}%'.format(future_prob * 100)
    probstring_future += ' (' + CRFtext(probstring_future, future_prob) + ')'
    ri_string_current = '{:.1f} years'.format(1.0/current_prob)
    ri_string_future = '{:.1f} years'.format(1.0/future_prob)
    if mode_is_ev:
        newHtml = '<br /><b>What EHE can you expect?</b><table style="width:600px; border: 1px solid black;"><tr style="border-bottom: 2px solid black; text-align: center; font-weight: bold"><td>year</td><td>magnitude</td><td>probability</td><td>recurrence interval</td><td>magnitude change</td><td>frequency change</td></tr>'
        newHtml += '<tr style="padding: 3px; text-align: center"><td>' + str(currentYear) + '</td><td>' + evstring_current + '</td><td>' + probstring_current + '</td><td>' + ri_string_current + '</td><td rowspan="2">' + changetext(current_ev, future_ev, False) + '</td><td rowspan="2">' + changetext(current_prob, future_prob, False) + '</td></tr>'
        newHtml += '<tr style="padding: 3px; text-align: center"><td>' + str(futureYear) + '</td><td>' + evstring_future + '</td><td>' + probstring_future + '</td><td>' + ri_string_current + '</td></tr></table>'
    else:
        newHtml = '<br /><b>How probable is EHE ≥ ' + str(persistent_threshold_value) + '?</b><table style="width:500px; border: 1px solid black;"><tr style="border-bottom: 2px solid black; text-align: center; font-weight: bold"><td>year</td><td>probability</td><td>recurrence interval</td><td>frequency change</td></tr>'
        newHtml += '<tr style="padding: 5px; text-align: center"><td>' + str(currentYear) + '</td><td style="text-align: center">' + probstring_current + '</td><td>' + ri_string_current + '</td><td rowspan="2">' + changetext(current_prob, future_prob, False) + '</td></tr>'
        newHtml += '<tr style="padding: 5px; text-align: center"><td>' + str(futureYear) + '</td><td style="text-align: center">' + probstring_future + '</td><td>' + ri_string_future + '</td></tr></table>'
    
    table_display.set_trait('value', newHtml)

def update_probplot():
# Replots plots of probability vs EHE threshold value
    probplot.canvas.draw()
    probplot.canvas.flush_events()
    global current_plot
    global future_plot
    if current_plot:
        dummy = current_plot.pop(0)
        dummy.remove()
    if future_plot:
        dummy = future_plot.pop(0)
        dummy.remove()
    current_data = renameBands(currentprobs).reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    current_plot = plt.plot([str(i) + 'd' for i in current_data.keys()], current_data.values(), label=str(currentYear), color='green')
    future_data = renameBands(futureprobs).reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom).getInfo()
    future_plot = plt.plot([str(i) + 'd' for i in future_data.keys()], future_data.values(), label=str(futureYear), color='lime')
    plt.legend()

In [12]:
@debounce(0.5)
def threshold_update(e):
    global persistent_threshold_value
    persistent_threshold_value = threshold_slider.value
    update_table()
    update_map()
    
@debounce(0.5)
def futureyear_update(e):
    global futureYear
    global futureprobs
    global prev_futureYear
    global persistent_ev_future
    futureYear = futureyear_slider.value
    futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()
    persistent_ev_future = get_ev_ehe(futureYear)
    update_probplot()
    update_table()
    update_map()
    prev_futureYear = futureYear
    
@debounce(0.5)
def marker_update(e):
    update_probplot()
    update_table()
    
@debounce(0.5)
def mode_update(e):
    global mode_is_ev
    global persistent_ev_current
    global persistent_ev_future
    mode_is_ev = mode_toggle.value == ev_val
    if mode_is_ev:
        if not persistent_ev_current:
            persistent_ev_current = get_ev_ehe(currentYear)
        if not persistent_ev_future:
            persistent_ev_future = get_ev_ehe(futureYear)
        threshold_slider.layout.visibility = 'hidden'
    else:
        threshold_slider.layout.visibility = 'visible'
    update_table()
    update_map()

def latlon_to_lonlat(coords):
# Converts EE coords (x, y) to ipyleaflet coords (y, x)
# Works for geojson coords
    if type(coords[0]) == float:
        return [coords[1], coords[0]]
    else:
        return [[[i[1],i[0]] for i in j] for j in coords]
    
def update_location(geom):
    global current_geom
    global current_location_marker
    Map.remove_layer(current_location_marker)
    draw_control.clear()
    new_geom = geom
    if new_geom['type'] == 'Point':
        current_geom = ee.Geometry.Point(new_geom['coordinates'])
        ipl_loc = latlon_to_lonlat(new_geom['coordinates'])
        current_location_marker = CircleMarker(location=latlon_to_lonlat(new_geom['coordinates']), name="location of interest")
        Map.add_layer(current_location_marker)
    else:
        current_geom = ee.Geometry.Polygon(new_geom['coordinates'])
        current_location_marker = Polygon(locations=latlon_to_lonlat(new_geom['coordinates']), name="location of interest")
        Map.add_layer(current_location_marker)
        
def drawcontrol_update(e):
    update_location(draw_control.last_draw['geometry'])
    update_probplot()
    update_table()

@debounce(0.5)
def uploadbutton_update(e):
    f = upload_button.data[-1]
    file_content = json.loads(codecs.decode(f, encoding="utf-8"))
    update_location(file_content['features'][0]['geometry'])
    polygon_coords = current_geom.coordinates().getInfo()[0]
    exes = np.array([i[0] for i in polygon_coords])
    whys = np.array([i[1] for i in polygon_coords])
    Map.setCenter(lat=np.mean(whys), lon=np.mean(exes))
    Map.set_trait('zoom', 9)
    update_probplot()
    update_table()

threshold_slider.observe(threshold_update)
futureyear_slider.observe(futureyear_update)
mode_toggle.observe(mode_update)
draw_control.observe(drawcontrol_update)
upload_button.observe(uploadbutton_update)

In [13]:
plt.ioff()

probplot = plt.figure(figsize=(5,4))
current_plot = None
future_plot = None
plt.xlabel('EHE ≥ threshold')
plt.ylabel('probability')

plt.ion();

In [14]:
update_probplot()
update_table()

In [15]:
# Dashboard layout
map_box = widgets.VBox([upload_button, Map], layout=Layout(width='600px', height='600px'))
slider_box = widgets.VBox([futureyear_slider, threshold_slider], layout=Layout(align_items='flex-start'))
hbar = widgets.HTML(value='<br/><div style="height:6px; width: 600px; background-color: #9090c0"></div>')
table_box = widgets.VBox([mode_toggle, table_display], layout=Layout(align_items='flex-start'))
middle_box = widgets.VBox([debug_display, slider_box, hbar, table_box])
items = [map_box, middle_box, probplot.canvas]
gridbox = widgets.GridBox(items, layout=Layout(grid_template_columns="repeat(3, 650px)"))

In [16]:
gridbox

GridBox(children=(VBox(children=(FileUpload(value={}, accept='.geojson', description='GeoJSON'), Map(center=[5…